In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [73]:
#Load Data
import pandas as pd
from sklearn.model_selection import train_test_split
train_data = pd.read_csv("../input/digit-recognizer/train.csv")
test_data = pd.read_csv("../input/digit-recognizer/test.csv")
X = train_data.drop(['label'],axis=1)
y = train_data['label']
train_X,val_X,train_y,val_y = train_test_split(X,y)

In [33]:
import numpy as np

In [74]:
#train_X = (np.array(train_X)).reshape(78,78,784)
train_X = np.array(train_X).reshape(31500,28,28,1)
val_X = np.array(val_X).reshape(10500,28,28,1)

In [75]:
train_X.shape

(31500, 28, 28, 1)

In [84]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Conv2D,Flatten,MaxPool2D
'''
Conv2D(128,4,activation='relu'),
        MaxPool2D(),
        Conv2D(64,4,activation='relu'),
        MaxPool2D(),
        Conv2D(32,4,activation='relu'),
'''
def classifier_model():
    return tf.keras.Sequential([
        Conv2D(128,(4,4),activation='relu',padding='same',input_shape=(28,28,1)),
        MaxPool2D(),        
        Conv2D(32,(3,3),activation='relu'),
        MaxPool2D(),  
        Conv2D(32,(3,3),activation='relu'),
        MaxPool2D(),  
        Flatten(),
        Dense(128,activation='relu'),
        Dense(10,activation='softmax')
    ])

In [85]:
model = classifier_model()
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_34 (Conv2D)           (None, 28, 28, 128)       2176      
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 12, 12, 32)        36896     
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 6, 6, 32)          0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 4, 4, 32)          9248      
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 2, 2, 32)          0         
_________________________________________________________________
flatten_14 (Flatten)         (None, 128)             

In [86]:


model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [87]:
model.fit(train_X,train_y,
    epochs=10,
    validation_data = (val_X,val_y)
)

Epoch 1/10
985/985 [==============================] - 40s 40ms/step - loss: 1.8094 - accuracy: 0.7788 - val_loss: 0.1376 - val_accuracy: 0.9604
Epoch 2/10
985/985 [==============================] - 39s 39ms/step - loss: 0.1185 - accuracy: 0.9641 - val_loss: 0.0852 - val_accuracy: 0.9740
Epoch 3/10
985/985 [==============================] - 39s 39ms/step - loss: 0.0739 - accuracy: 0.9769 - val_loss: 0.0803 - val_accuracy: 0.9757
Epoch 4/10
985/985 [==============================] - 38s 39ms/step - loss: 0.0606 - accuracy: 0.9824 - val_loss: 0.0888 - val_accuracy: 0.9754
Epoch 5/10
985/985 [==============================] - 38s 39ms/step - loss: 0.0624 - accuracy: 0.9810 - val_loss: 0.0886 - val_accuracy: 0.9759
Epoch 6/10
985/985 [==============================] - 38s 39ms/step - loss: 0.0449 - accuracy: 0.9858 - val_loss: 0.0881 - val_accuracy: 0.9764
Epoch 7/10
985/985 [==============================] - 38s 39ms/step - loss: 0.0380 - accuracy: 0.9882 - val_loss: 0.0763 - val_accuracy:

In [89]:
test_data.shape

(28000, 784)

In [90]:
test_X = np.array(test_data).reshape(28000,28,28,1)

In [94]:
pred = model.predict(test_X)

In [96]:
pred = np.argmax(pred,axis=1)

In [102]:
submission = pd.DataFrame()
submission['Label'] = pred
submission['ImageId'] = submission.index+1

In [103]:
submission = submission.reindex(columns=['ImageId','Label'])

In [106]:
submission.to_csv('submit.csv',index=False)

Traceback (most recent call last):
  File "/opt/conda/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/opt/conda/lib/python3.7/site-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/opt/conda/lib/python3.7/site-packages/kaggle/api/kaggle_api_extended.py", line 166, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
